In [1]:
!pip install torch torchtext
!git clone https://github.com/neubig/nn4nlp-code.git

    100% |████████████████████████████████| 484.0MB 24kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x5bf5c000 @  0x7fc0e503f1c4 0x46d6a4 0x5fcbcc 0x4c494d 0x54f3c4 0x553aaf 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54e4c8
    100% |████████████████████████████████| 51kB 12.8MB/s 
    100% |████████████████████████████████| 51kB 13.1MB/s 
  Running setup.py bdist_wheel for torchtext ... - \ done
  Stored in directory: /content/.cache/pip/wheels/42/a6/f4/b267328bde6bb680094a0c173e8e5627ccc99543abded97204
Successfully built torchtext
fatal: destination path 'nn4nlp-code' already exists and is not an empty directory.


In [0]:
from collections import defaultdict
import math
import time
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [0]:
N=2 #length of window on each side (so N=2 gives a total window size of 5, as in t-2 t-1 t t+1 t+2)
EMB_SIZE = 128 # The size of the embedding

embeddings_location = "embeddings.txt" #the file to write the word embeddings to
labels_location = "labels.txt" #the file to write the labels to

# We reuse the data reading from the language modeling class
w2i = defaultdict(lambda: len(w2i))
S = w2i["<s>"]
UNK = w2i["<unk>"]
def read_dataset(filename):
  with open(filename, "r") as f:
    for line in f:
      yield [w2i[x] for x in line.strip().split(" ")]

In [0]:
# Read in the data
train = list(read_dataset("nn4nlp-code/data/ptb/train.txt"))
w2i = defaultdict(lambda: UNK, w2i)
dev = list(read_dataset("nn4nlp-code/data/ptb/valid.txt"))
i2w = {v: k for k, v in w2i.items()}
nwords = len(w2i)

with open(labels_location, 'w') as labels_file:
  for i in range(nwords):
    labels_file.write(i2w[i] + '\n')


In [0]:
class CBOW(nn.Module):
  def __init__(self, vocab_size, embed_dim):
    super(CBOW, self).__init__()
    
    self.embeddings_bag = nn.EmbeddingBag(vocab_size, embed_dim, mode='sum')
    self.fcl = nn.Linear(embed_dim, vocab_size, bias=False)
    
  def forward(self, x):
    x = self.embeddings_bag(x.view(1, -1))
    return self.fcl(x)

In [0]:
model = CBOW(nwords, EMB_SIZE)
loss_fn = nn.CrossEntropyLoss()
opt = torch.optim.SGD(model.parameters(), lr=0.1)

In [0]:
# Calculate the loss value for the entire sentence
def calc_sent_loss(sent):
  #add padding to the sentence equal to the size of the window
  #as we need to predict the eos as well, the future window at that point is N past it 
  padded_sent = [S] * N + sent + [S] * N

  # Step through the sentence
  all_losses = [] 
  for i in range(N,len(sent)+N):
    model.zero_grad()
    
    logits = model(torch.LongTensor(padded_sent[i-N:i] + padded_sent[i+1:i+N+1]))
    loss = F.cross_entropy(logits, torch.tensor(padded_sent[i]).view(1))
    loss.backward()
    opt.step()
    all_losses.append(loss.cpu().detach().numpy())
  return sum(all_losses)

In [0]:
MAX_LEN = 100

for ITER in range(100):
  print("started iter %r" % ITER)
  # Perform training
  random.shuffle(train)
  train_words, train_loss = 0, 0.0
  start = time.time()
  for sent_id, sent in enumerate(train):
    my_loss = calc_sent_loss(sent)
    train_loss += my_loss
    train_words += len(sent)
#     my_loss.backward()
#     trainer.update()
    if (sent_id+1) % 5000 == 0:
      print("--finished %r sentences" % (sent_id+1))
  print("iter %r: train loss/word=%.4f, ppl=%.4f, time=%.2fs" % (ITER, train_loss/train_words, math.exp(train_loss/train_words), time.time()-start))
  # Evaluate on dev set
  dev_words, dev_loss = 0, 0.0
  start = time.time()
  for sent_id, sent in enumerate(dev):
    my_loss = calc_sent_loss(sent)
    dev_loss += my_loss
    dev_words += len(sent)
#     trainer.update()
  print("iter %r: dev loss/word=%.4f, ppl=%.4f, time=%.2fs" % (ITER, dev_loss/dev_words, math.exp(dev_loss/dev_words), time.time()-start))

  print("saving embedding files")
  with open(embeddings_location, 'w') as embeddings_file:
    W_w_np = W_w_p.as_array()
    for i in range(nwords):
      ith_embedding = '\t'.join(map(str, W_w_np[i]))
      embeddings_file.write(ith_embedding + '\n')

started iter 0
--finished 5000 sentences
--finished 10000 sentences
--finished 15000 sentences
--finished 20000 sentences
--finished 25000 sentences
--finished 30000 sentences
--finished 35000 sentences
